In [142]:
aa_map = pd.read_csv('../src_data/aa_chart.csv', index_col=None)
str(list(aa_map['Function']))

"['Aromatic', 'Aromatic', 'Aromatic', 'Hydrophobic', 'Hydrophobic', 'Hydrophobic', 'Hydrophobic', 'Hydrophobic', 'Polar', 'Polar', 'Polar', 'Polar', 'Proline', 'Glycine', 'Charge (-)', 'Charge (-)', 'Charge (+)', 'Charge (+)', 'Charge (+)', 'Excluded']"

In [172]:
# @TODO Dynamically filter peptide set based on length(s) of input sequences of binders
#       i.e. 2 binders, one 11 AA long, one 13 AA long, each gets their own "subset" of the
#       full peptide lilst that can be compared to it. For any number of input sequences

# @TODO implement method for both similarities to M6 and GrBP5 to interrelate and act as their own feature set:
# i.e. if a peptide matches both peptides in temrs of sequence at some index, that should be important rather than
# having it equal to one matching only one

# @TODO: Maybe as originally planned implement binders inputted as dictionary of multiple values, each with separate
#      dataframes within the same class --> for big similarity calculations. Or just create multiple SequenceSimilarity instances

'''
Class to calculate several simialrity metrics for an input list of peptides given a sequence string to compare it to.
USAGE: 
1. Create SequenceSimilarityObject({sequence string}, {dictionary of data paths} (to be deprecated soon -- use
   internal class data), {path to peptides csv}, {column title of sequence values of aforementioned peptides csv})
   for any number of sequences you want to be compared to the list of peptides
2. For each object, call object.generate_similarity() to fill out the Dataframe with similarity metrics
3. To whittle down this similarity matrix to list only those peptides with pattern matching of a minimum length
   at a matching index in the rerence peptide (henceforth the binder), call object.get_df_with_binder_subseqs(min_length={#})
@ Author: Chris Pecunies, with help from Savvy Gupta and Aaron Tsang
@ Date: February 12, 2020
'''
import pandas as pd
import numpy as np
from scipy.stats import kendalltau
import matplotlib.pyplot as plt
from typing import Set, Tuple, Dict, List

class SequenceSimilarity:
    '''
    Class that takes in a path to a list of amino acid sequences as well
    as any number of peptide sequences explicitly that are known to have
    a certain set of properties. Generates metrics for similarity for each
    peptide in path and returns domains AA sequence with high similarity
    '''

    def __init__(self, binder: str,   
                 data_paths: Dict,     #@TODO Make another .py file containing object version of necessary sim matrices/conversions, etc.
                 peps_path: str,       #      to reduce reliance on outside data
                 aa_col: str):         #-> The column in peps_path csv where sequences are held
        
        self.AA = 'FYWAVILMSTNQPGDEKHRC'
        self.NUM = [0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 4, 5, 5, 6, 6, 6, 7]
        self.EIIP = [0.0946, 0.0516, 0.0548, 0.0373, 0.0057, 0.0, 0.0, 0.0823,\
                     0.0829, 0.0941, 0.0036, 0.0761, 0.0198, 0.005, 0.1263, 0.0058, \
                     0.0371, 0.0242, 0.0959, 0.0829]
        self.FNS = ['Aromatic', 'Aromatic', 'Aromatic', 'Hydrophobic', 'Hydrophobic', \
                    'Hydrophobic', 'Hydrophobic', 'Hydrophobic', 'Polar', 'Polar', 'Polar', \
                    'Polar', 'Proline', 'Glycine', 'Charge (-)', 'Charge (-)', 'Charge (+)', \
                    'Charge (+)', 'Charge (+)', 'Excluded']
        
        self.binder = binder   # to get binder_len, just use len(self.binders)
        self.__read_similarity_data(data_paths) # !TO BE DEPRECATED! Just store data in class

        self.aa_col = aa_col
        self.columns = ['EIIP_Seq', 'NUM_Seq', 'PAM30', 'BLOSUM', 'RRM_SN', 'RRM_Corr']
        ## @TODO: Add "# of matching sseqs, cross entropy AA, cross entropy Num columns ?"
        
        self.peps = pd.read_csv(peps_path)
        self.peps.columns = [aa_col]
        self.peps_same_len = self.peps[self.peps[aa_col].str.len() == len(binder)]
        if len(self.peps_same_len) == 0:
            raise Exception("No peptides of same length as binder found")
        self.peps_sl_sim = self.peps_same_len.copy()
        for col in self.columns:
            self.peps_sl_sim[col] = None
        
        #self.get_similarities()

    def __read_similarity_data(self, data_path_dict) -> None:
        """
        Private method to store the paths of any data needed
        for similarity calcs and create Dataframes from them
        """
        self.data = dict.fromkeys(data_path_dict.keys())
        for data in data_path_dict.keys():
            self.data[data] = pd.read_csv(data_path_dict[data], index_col=0)

            
    def _update_AA_conversion(self, conv_type: str = None) -> None:
        AA_map = self.aa_dict(conv_type)
        AA_conv = lambda pep: [AA_map[AA] if AA in self.AA else 0 for AA in pep]
        
        self.peps_sl_sim[conv_type+"_Seq"] = self.peps_sl_sim[self.aa_col].apply(AA_conv)

    def _update_PAM30_similarity(self) -> None:
        pass

    def _update_BLOSUM_similarity(self) -> None:
        pass

    def _update_RRM_SN_similarity(self) -> None:
        pass

    def _update_RRM_corr_similarity(self) -> None:
        pass

    def _update_matching_sseqs(self):
        pass
    
    def update_similarities(self) -> None:
        
        self._update_AA_conversion(conv_type='EIIP')
        self._update_AA_conversion(conv_type='NUM')
        self._update_BLOSUM_similarity()
        self._update_PAM30_similarity()
        self._update_RRM_SN_similarity()
        self._update_RRM_corr_similarity()
        self._update_matching_sseqs()

    def df_filter_subseq(self, sub_seq: str, ind: int = None) -> pd.DataFrame:
        '''
        Takes in a subsequence of equal or lesser length to
        peptides in class peptide dataframe and returns a dataframe
        containing only those peptides containing the sequence
        '''
        if not {*sub_seq}.issubset({*self.AA}):
            raise Exception('Invalid subsequence')
        if ind is None:
            return self.peps_sl_sim[self.peps_sl_sim[self.aa_col].str.contains(sub_seq)]
        return self.peps_sl_sim[self.peps_sl_sim[self.aa_col].str.find(sub_seq) == ind]

    def get_sim_matrix(self, seq) -> pd.DataFrame:
        return self.data.filter

    def get_binder_subseq(self) -> pd.DataFrame: #Each binder associated with list of (sseq, index)
        '''
        Generates all possible subsequences for binders. Returns in dict, where
        each binder corresponds to a dictionary of the index where it occurs,
        and the subsequence that occurs
        '''
        return [(self.binder[i:j], i) for i in range(len(self.binder)) for j in range(i+1, len(self.binder)+1)]


    def get_df_with_binder_subseqs(self, min_length: int = 0) -> Dict[str, pd.DataFrame]:
        '''
        Returns a filtered version of self.peps_same_len DataFrame containing only
        those rows with sequences which contain subsequences (of min length specified in parameter) 
        of the two binder sequences in the locations where they occur in the binders
        '''
        sseq = self.get_binder_subseq()
        filtered_data = [self.df_fitler_subseq(ss,i) for (ss,i) in sseq[binder] if len(ss) >= min_length]
        return pd.concat(filtered_data)
    
    #------------------helper methods ------------------------------------#
    
    def aa_dict(self, conversion) -> Dict:
        if conversion == 'NUM':
            return dict(zip(self.AA, self.NUM))
        elif conversion == 'EIIP':
            return dict(zip(self.AA, self.EIIP))
        elif conversion == 'FNS':
            return dict(zip(self.AA, self.FNS))
        else:
            raise Exception("Must specify AA conversion target")
                
                
    #-------------------miscellaneous methods------------------------------#

    def get_kendalltau_corr_map(self) -> Tuple:
        return kendalltau(self.data['AA_MAP'][['Num']], self.data['AA_MAP'][['EIIP']])

In [173]:
DATA_PATHS = {
        "BLOSUM":"../src_data/BLOSUM.csv",
        "PAM30":"../src_data/pam30.csv",
        "AA_MAP":"../src_data/aa_chart.csv",
}
SEQS = {
    'GRBP5':'IMVTESSDYSSY',
    'M6':'IMVTASSAYDDY'
}
AA_COL = 'Sequences'
PEP_PATH = '../src_data/Sequence_data.csv'
grbp5_sim = SequenceSimilarity(SEQS['GRBP5'], DATA_PATHS, PEP_PATH, AA_COL)
m6_sim = SequenceSimilarity(SEQS['M6'], DATA_PATHS, PEP_PATH, AA_COL)

    # --------------------------- debug
    # Check to make sure df filter works
grbp5_sim.df_filter_subseq('SVP', ind=0)
grbp5_sim.update_similarities()
#filtered_grbp5 = grbp5sim.get_df_with_binder_subseqs()
m6_sim.df_filter_subseq('SVP', ind=0)
m6_sim.update_similarities()
#filtered_m6 = m6.get_df_with_binder_subseqs()

In [176]:
m6_sim.peps_sl_sim

,Sequences,EIIP_Seq,NUM_Seq,PAM30,BLOSUM,RRM_SN,RRM_Corr
13,SVPHFSDEDKDP,"[0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...","[2, 1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 3]",None,None,None,None
14,VPHFSDEDKDPE,"[0.0057, 0.0198, 0.0242, 0.0946, 0.0829, 0.126...","[1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 3, 5]",None,None,None,None
28,SVPHFSEEEKEA,"[0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...","[2, 1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 1]",None,None,None,None
29,VPHFSEEEKEAE,"[0.0057, 0.0198, 0.0242, 0.0946, 0.0829, 0.005...","[1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 1, 5]",None,None,None,None
43,SVPHFSDEDKDP,"[0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...","[2, 1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 3]",None,None,None,None
...,...,...,...,...,...,...,...
28873,FLRRIRPKLKWD,"[0.0946, 0.0, 0.0959, 0.0959, 0.0, 0.0959, 0.0...","[0, 1, 6, 6, 1, 6, 3, 6, 1, 6, 0, 5]",None,None,None,None
28874,LRRIRPKLKWDN,"[0.0, 0.0959, 0.0959, 0.0, 0.0959, 0.0198, 0.0...","[1, 6, 6, 1, 6, 3, 6, 1, 6, 0, 5, 2]",None,None,None,None
28875,RRIRPKLKWDNQ,"[0.0959, 0.0959, 0.0, 0.0959, 0.0198, 0.0371, ...","[6, 6, 1, 6, 3, 6, 1, 6, 0, 5, 2, 2]",None,None,None,None
28903,YGGFLRRQFKVV,"[0.0516, 0.005, 0.005, 0.0946, 0.0, 0.0959, 0....","[0, 4, 4, 0, 1, 6, 6, 2, 0, 6, 1, 1]",None,None,None,None


In [175]:
grbp5_sim.update_similarities()

In [110]:
print(type(similarity.peps))
peps = pd.read_csv('../src_data/Sequence_data.csv')
peps.columns = ['Sequences']

peps

<class 'dict'>


,Sequences
0,FSEEEKEPE
1,SVPHFSDED
2,VPHFSDEDK
3,PHFSDEDKD
4,HFSDEDKDP
...,...
28910,VMGHFRWDR
28911,MGHFRWDRF
28912,YVMGHFRWDR
28913,VMGHFRWDRF


# Get RRM S/N similarity

In [12]:
def get_RRM():
    data = similarity.peps_sl_sim
    matrix = similarity.data['PA]
    print(data.head(), len(data))
    
get_RRM()

       Sequences                                           EIIP_Seq  \
13  SVPHFSDEDKDP  [0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...   
14  VPHFSDEDKDPE  [0.0057, 0.0198, 0.0242, 0.0946, 0.0829, 0.126...   
28  SVPHFSEEEKEA  [0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...   
29  VPHFSEEEKEAE  [0.0057, 0.0198, 0.0242, 0.0946, 0.0829, 0.005...   
43  SVPHFSDEDKDP  [0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...   

                                 Num_Seq PAM30 BLOSUM RRM_SN RRM_Corr  
13  [2, 1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 3]  None   None   None     None  
14  [1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 3, 5]  None   None   None     None  
28  [2, 1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 1]  None   None   None     None  
29  [1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 1, 5]  None   None   None     None  
43  [2, 1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 3]  None   None   None     None   3079


# Get BLOSUM similarity

In [29]:
def get_BLOSUM_similarity():
    data = similarity.peps_sl_sim
    binders = similarity.binders
    BLOSUM = similarity.data['BLOSUM']
    for binder in binders:
        for pep in data['Sequences']:
            dist = 0
            for aa1 in pep:
                for aa2 in binder:
                    dist += BLOSUM[aa1].loc[aa2]
                    
    print(data['Sequences'])
    print(binders)
    print(BLOSUM)
    
#get_BLOSUM_similarity()
similarity.data['BLOSUM']

,A,R,N,D,C,Q,E,G,H,I,...,P,S,T,W,Y,V,B,Z,X,*
A,4,-1,-2,-2,0,-1,-1,0,-2,-1,...,-1,1,0,-3,-2,0,-2,-1,0,-4
R,-1,5,0,-2,-3,1,0,-2,0,-3,...,-2,-1,-1,-3,-2,-3,-1,0,-1,-4
N,-2,0,6,1,-3,0,0,0,1,-3,...,-2,1,0,-4,-2,-3,3,0,-1,-4
D,-2,-2,1,6,-3,0,2,-1,-1,-3,...,-1,0,-1,-4,-3,-3,4,1,-1,-4
C,0,-3,-3,-3,9,-3,-4,-3,-3,-1,...,-3,-1,-1,-2,-2,-1,-3,-3,-2,-4
Q,-1,1,0,0,-3,5,2,-2,0,-3,...,-1,0,-1,-2,-1,-2,0,3,-1,-4
E,-1,0,0,2,-4,2,5,-2,0,-3,...,-1,0,-1,-3,-2,-2,1,4,-1,-4
G,0,-2,0,-1,-3,-2,-2,6,-2,-4,...,-2,0,-2,-2,-3,-3,-1,-2,-1,-4
H,-2,0,1,-1,-3,0,0,-2,8,-3,...,-2,-1,-2,-2,2,-3,0,0,-1,-4
I,-1,-3,-3,-3,-1,-3,-3,-4,-3,4,...,-3,-2,-1,-3,-1,3,-3,-3,-1,-4


# Get PAM30 similiarity

# REVELATION
What if... maybe... instead of handling binder as a dict, handle AT MOST one input string as the binder sequence so there is a one:one correspondence between the class and the sequence for similarity calculations???